In [1]:
from io import StringIO
import sagemaker
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sagemaker.pytorch import PyTorch
import os

boto_session = boto3.Session(region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

role = "arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928"

# Define your bucket name and file key (file path in S3)
BUCKET_NAME = "blue-blood-data"
FILE_KEY = "final_df.csv"  # Change to your actual file path in S3

# Create an S3 client
s3 = boto3.client("s3")

# Fetch the file from S3
response = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)

# Read the CSV file into a pandas DataFrame
csv_content = response["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(csv_content))

/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/26/25 17:10:43] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=968082;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=264000;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ubaid/Library/Application Support/sagemaker/config.yaml


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=406691;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=640932;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=657656;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=414767;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

[03/26/25 17:10:44] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=755212;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=345659;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

In [2]:
df.head()

,subject_id,prescription_start,prescription_rx_embeddings,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,[ 3.5185558e-01 1.2351961e-01 -1.2304356e-01 ...,0.010317,3,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10013,2125-10-05T00:00:00,[ 0.45182744 0.3218944 -0.5210766 0.315588...,0.011905,4,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10013,2125-10-05T00:00:00,[ 4.5976555e-01 1.9232908e-01 -5.7382131e-01 ...,0.009921,2,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10013,2125-10-05T00:00:00,[ 6.26637757e-01 2.61670560e-01 -2.40684357e-...,0.017857,3,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10013,2125-10-05T00:00:00,[ 4.15423244e-01 -1.28793076e-01 -2.01883331e-...,0.020833,3,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import pandas as pd
import numpy as np
import re

def clean_and_convert(x):
    if isinstance(x, np.ndarray):  # If already a NumPy array, return as-is
        return x
    if isinstance(x, str):  # Only process strings
        try:
            x = re.sub(r'[\[\]]', '', x)  # Remove square brackets
            cleaned = re.sub(r'\s+', ' ', x.strip())  # Remove extra spaces
            return np.array([float(i) for i in cleaned.split(' ')])  # Convert to NumPy array
        except Exception as e:
            return x  # Return original value in case of error
    return x  # If NaN or unexpected type, return as-is

# Check the values before applying the function
print(df['prescription_rx_embeddings'].head())

# Apply the function to the 'prescription_rx_embeddings' column
df['prescription_rx_embeddings'] = df['prescription_rx_embeddings'].apply(clean_and_convert)

# Check the result
df['prescription_rx_embeddings'][0]

0    [ 3.5185558e-01  1.2351961e-01 -1.2304356e-01 ...
1    [ 0.45182744  0.3218944  -0.5210766   0.315588...
2    [ 4.5976555e-01  1.9232908e-01 -5.7382131e-01 ...
3    [ 6.26637757e-01  2.61670560e-01 -2.40684357e-...
4    [ 4.15423244e-01 -1.28793076e-01 -2.01883331e-...
Name: prescription_rx_embeddings, dtype: object


array([ 3.5185558e-01,  1.2351961e-01, -1.2304356e-01, -3.3200896e-01,
       -1.0614018e+00,  1.9121327e-01,  2.7952591e-01,  1.0901182e-02,
       -4.5522165e-01, -6.9268858e-01,  1.5272896e-01, -6.2518787e-01,
       -5.0267065e-01,  2.3799901e-01,  3.2225102e-01,  3.5973153e-01,
       -8.7883368e-02, -6.6698366e-01,  1.2195622e+00,  3.2569757e-01,
       -9.7169526e-02, -9.6203506e-01,  7.8181475e-01,  3.1520061e+00,
        1.0999583e+00,  8.8330621e-01, -1.3076260e+00, -1.1109836e+00,
       -2.1281254e+00,  5.1991540e-01, -1.2142467e+00, -9.5872365e-02,
       -2.3115242e-01,  9.2357832e-01,  7.1854466e-01, -4.1259676e-01,
       -7.4190390e-01,  1.6999608e+00,  4.8031932e-01,  8.1066114e-01,
       -1.0656835e+00, -2.8141283e-03, -5.6771725e-01,  4.5298275e-01,
        2.2276033e-01,  4.5889392e-02, -2.4509761e-01, -1.4576931e-01,
        6.5086001e-01,  1.1289541e-01,  4.4198048e-01, -8.2376510e-01,
       -4.1519284e-01,  4.0492430e-01,  1.8494433e-01, -5.1815975e-01,
      

In [4]:
df['prescription_rx_embeddings'][0].shape

(128,)

In [5]:
# function that gets the unique pairs of subject_id and prescription_start
def get_unique_pairs(df):
    subject_ids = df['subject_id'].unique()
    patient_date_pairs = {id: set() for id in subject_ids}

    for subj in subject_ids:
        df[df['subject_id'] == subj]['prescription_start'].apply(lambda x: patient_date_pairs[subj].add(x))
        # convert set to list
        patient_date_pairs[subj] = list(patient_date_pairs[subj])
    return patient_date_pairs

In [14]:
# function that gets the prescription array with the proper format
def get_presc_input(df):
    prescriptions = []
        
    # Iterate through rows of the DataFrame
    for _, row in df.iterrows():
        # Extract values from each row
        presc = row['prescription_rx_embeddings']
        dose_val = row['prescription_dose_val_rx']
        dose_unit = row['prescription_dose_unit_rx']
        
        # Concatenate the prescription embedding with the dose value and unit
        combined = np.concatenate((presc, np.array([dose_val, dose_unit])))
        prescriptions.append(combined)
    
    # Convert list to numpy array
    prescriptions = np.array(prescriptions)
    print(prescriptions.shape)

    return prescriptions

In [24]:
# function that adds the proper padding to our input arrays
def add_padding(prescriptions, pre_treatment, post_treatment):
    # reshape pre_treatment and post_treatment to be 2D arrays
    pre_treatment = pre_treatment.reshape(1, -1)
    post_treatment = post_treatment.reshape(1, -1)
    
    # Pad or truncate to 20 rows
    if prescriptions.shape[0] < 20:
        # Pad with zeros to reach 20 rows
        prescriptions = np.pad(prescriptions, ((0, 20 - prescriptions.shape[0]), (0, 0)), mode='constant')
    elif prescriptions.shape[0] > 20:
        # Truncate to 20 rows
        prescriptions = prescriptions[:20, :]
    
    # Pad pre_treatment and post_treatment to 20 rows
    pre_treatment = np.pad(pre_treatment, ((0, 19), (0, 0)), mode='constant')  # pad to (20, 25)
    post_treatment = np.pad(post_treatment, ((0, 19), (0, 0)), mode='constant')  # pad to (20, 25)
    
    # Now pad columns to reach 180 features for each
    padded_prescriptions = np.pad(prescriptions, ((0, 0), (0, 0)), mode='constant')
    padded_pre_treatment = np.pad(pre_treatment, ((0, 0), (0, 105)), mode='constant')
    padded_post_treatment = np.pad(post_treatment, ((0, 0), (0, 105)), mode='constant') 
    
    return padded_prescriptions, padded_pre_treatment, padded_post_treatment

In [ ]:
# function that does the good stuff
def train_model(df, model):
    # Get unique patient/date pairs
    patient_date_pairs = get_unique_pairs(df)
    # Iterate through the patient/date pairs
    for patient in patient_date_pairs:
        for date in patient_date_pairs[patient]:
            # Get the data for the current patient/date pair
            patient_data = df[(df['subject_id'] == patient) & (df['prescription_start'] == date)]

            # drop unnecessary columns
            patient_data = patient_data.drop(['subject_id', 'prescription_start', 'pre_charttime', 'post_charttime'], axis=1)
            
            # Get the prescription input for the current patient/date pair (2DArray with shape (num_prescriptions, 130))
            prescriptions = get_presc_input(patient_data)

            # pre_treatment and post_treatment are 1D arrays
            pre_treatment = np.array(patient_data[[col for col in patient_data.columns if col.startswith('pre_')]].values[0])
            post_treatment = np.array(patient_data[[col for col in patient_data.columns if col.startswith('post_')]].values[0])

            print("prescriptions shape:", prescriptions.shape)
            print("pre_treatment shape:", pre_treatment.shape)
            print("post_treatment shape:", post_treatment.shape)

            # add padding to the inputs
            padded_prescriptions, padded_pre_treatment, padded_post_treatment = add_padding(prescriptions, pre_treatment, post_treatment)

            # Create the full sequence (1 patient, 3 time steps, 180 features)
            X_train = np.array([[
                padded_pre_treatment,           # Time Step 1: Pre-Treatment
                padded_prescriptions,           # Time Step 2: Prescription
                padded_post_treatment           # Time Step 3: Post-Treatment
            ]])

            y_train = X_train[:, -1, :]  # Target is the last time step (Post-Treatment)

            # Check data shape
            print("X_train shape:", X_train.shape)
            print("y_train shape:", y_train.shape)

            # Train the model
            model.fit(X_train, y_train, epochs=50, batch_size=1)

            

In [25]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Define LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(3, 20, 130)),  # 3 time steps, 20 rows, 130 features
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation="relu"),
    Dense(130)  # Predicting post-treatment blood profile (25 values)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Print model summary
model.summary()

# Uncomment to train the model when ready
train_model(df, model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 from keras.layers import LSTM, Dense, Dropout                                               │
│    4                                                                                             │
│    5 # Define LSTM model                                                                         │
│ ❱  6 model = Sequential([                                                                        │
│    7 │   LSTM(64, return_sequences=True, input_shape=(3, 20, 130)),  # 3 time steps, 20 rows,    │
│    8 │   Dropout(0.2),                                                                           │
│    9 │   LSTM(32, return_sequences=False),                                                       │
│                                                                                                  │
│ /opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/keras/src/models/sequential.py:76 in     │
│ __init__                                                                                         │
│                                                                                                  │
│    73 │   │   if layers:                                                                         │
│    74 │   │   │   for layer in layers:                                                           │
│    75 │   │   │   │   self.add(layer, rebuild=False)                                             │
│ ❱  76 │   │   │   self._maybe_rebuild()                                                          │
│    77 │                                                                                          │
│    78 │   def add(self, layer, rebuild=True):                                                    │
│    79 │   │   """Adds a layer instance on top of the layer stack.                                │
│                                                                                                  │
│ /opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/keras/src/models/sequential.py:141 in    │
│ _maybe_rebuild                                                                                   │
│                                                                                                  │
│   138 │   │   self._functional = None                                                            │
│   139 │   │   if isinstance(self._layers[0], InputLayer) and len(self._layers) > 1:              │
│   140 │   │   │   input_shape = self._layers[0].batch_shape                                      │
│ ❱ 141 │   │   │   self.build(input_shape)                                                        │
│   142 │   │   elif hasattr(self._layers[0], "input_shape") and len(self._layers) > 1:            │
│   143 │   │   │   # We can build the Sequential model if the first layer has the                 │
│   144 │   │   │   # `input_shape` property. This is most commonly found in Functional            │
│                                                                                                  │
│ /opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/keras/src/layers/layer.py:226 in         │
│ build_wrapper                                                                                    │
│                                                                                                  │
│    223 │   │   def build_wrapper(*args, **kwargs):                                               │
│    224 │   │   │   with obj._open_name_scope():                                                  │
│    225 │   │   │   │   obj._path = current_path()                                                │
│ ❱  226 │   │   │   │   original_build_method(*args, **kwarg

In [11]:
# # Test patient (pre-treatment + prescription, but post-treatment is missing)
# X_test = np.array([[
#     [12.5, 4500, 13.2, 4.5, 140],  # Pre-Treatment
#     [0.0, 1.0, 500, 0.0, 0.0],     # Prescription (padded to 5 features)
#     [0, 0, 0, 0, 0]  # Unknown post-treatment (to be predicted)
# ]])

# # Predict post-treatment
# predicted_post_treatment = model.predict(X_test)
# print("Predicted Post-Treatment Blood Profile:", predicted_post_treatment)